## Question-1

In [1]:
import numpy as np
import copy, sys
import pandas as pd

roll_no_last = 4

def calculate_f(x, weights, y):
    return ((np.dot(x,weights) - y)**2).sum() / (2 * x.shape[0])

def grad_f(x, weights, y):
    first_term = (x[:,:-1] * (np.dot(x,weights) - y)).sum()/ x.shape[0]
    second_term = (np.dot(x,weights) - y).sum() / x.shape[0]
    return np.array([[first_term],[second_term]],dtype=np.float64)

def check_armijo_wolf_cond(w_k, d_k, x,y, alpha, beta1, beta2):
    xk_alp_dk = w_k + alpha * d_k
    f_grad = grad_f(x, w_k, y)
    f_grad_alph = grad_f(x, xk_alp_dk, y)
    armijo_left = calculate_f(x, xk_alp_dk, y)
    armijo_right = calculate_f(x, w_k, y) + alpha * beta1*np.dot(f_grad.T,d_k)
    wolf_left = np.dot(f_grad_alph.T, d_k)
    wolf_right = beta2 * np.dot(f_grad.T,d_k)
    return (armijo_left <= armijo_right) and (wolf_left >= wolf_right)

def steepest_direction(w_k, d_k, x, y,beta1, beta2, r):
    alpha = 1
    while not check_armijo_wolf_cond(w_k, d_k, x, y, alpha, beta1, beta2):
        alpha = alpha * r
    return alpha

def find_w1(w_0, x, y, beta1, beta2,epsilon, r):
    w_k = w_0
    d_k = -grad_f(x, w_k, y)
    norm_grad = np.linalg.norm(-d_k)
    iterations = 0
    while norm_grad > epsilon:
        alpha = steepest_direction(w_k, d_k, x, y, beta1, beta2, r)
        w_k = w_k + alpha * d_k
        d_k = -grad_f(x, w_k, y)
        norm_grad = np.linalg.norm(-d_k)
        iterations += 1
        if iterations == 1000:
            break
    return w_k, iterations

beta1 = 1e-4
beta2 = 0.9
epsilon = 0.01
r = 0.5
w_0 = np.array([[0],[0]], dtype=np.float64)
data = pd.read_csv("2_column_dataset[linear].csv")
data = np.array(data)
x = data[:,:-1]
y = data[:,-1:]
x = np.hstack([x,np.ones(x.shape)])
w, iterations = find_w1(w_0, x, y, beta1, beta2, epsilon, r)
# print('\n')
print(w)
print(f"total_iterations: {iterations}")
print(calculate_f(x, w, y))

FileNotFoundError: [Errno 2] No such file or directory: '2_column_dataset[linear].csv'

### Question-2

In [100]:
def return_dataset(x, y, batch_size):
    total_batches = int(x.shape[0] / batch_size)
    remaining_points = x.shape[0] - total_batches * batch_size
    for batch in range(total_batches):
        yield (x[batch:batch+batch_size],y[batch:batch+batch_size])
    if remaining_points != 0:
        yield (x[-remaining_points:],y[-remaining_points:])

In [101]:
def find_w2(w_k, x, y, iteration):
    d_k = -grad_f(x, w_k, y)
    alpha = 1 / (iteration + 1)
    w_k = w_k + alpha * d_k
    return w_k

In [102]:
beta1 = 1e-4
beta2 = 0.9
epsilon = 0.001
r = 0.5
batch_size = 1
w_0 = np.array([[0],[0]], dtype=np.float64)
x = data[:,:-1]
y = data[:,-1:]
x = np.hstack([x,np.ones(x.shape)])
data_loader = return_dataset(x, y, batch_size)
iteration = 0

In [103]:
while calculate_f(x, w_0, y) > 2.003:
    for idx, (dx,dy) in enumerate(data_loader):
        cur_w = find_w2(w_0, dx, dy, iteration)
        iteration += 1
        if calculate_f(x, cur_w, y) < 2.003:
            w_0 = cur_w
            break
        if calculate_f(x, cur_w, y) < calculate_f(x, w_0, y):
            w_0 = cur_w
print(f"total_iterations: {iteration}")
print(w_0)

total_iterations: 97
[[3.66670413]
 [0.08151968]]


## Question-3

In [104]:
def cal_cost(theta,X,y):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    return cost
def minibatch_gradient_descent(X,y,theta,learning_rate=0.01, iterations=10, batch_size =10):
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)
    for it in range(iterations):
        cost =0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]
            X_i = np.c_[np.ones(len(X_i)),X_i]
            prediction = np.dot(X_i,theta)
            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost
    return theta, cost_history

In [105]:
lr = 0.05
n_iter = 1000
theta = np.random.randn(3, 1)
theta,cost_history = minibatch_gradient_descent(x,y,theta,lr,n_iter)
print('Theta0:          {:0.3f},\nTheta1:          {:0.3f}'.format(theta[0][0],theta[1][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

Theta0:          0.809,
Theta1:          3.671
Final cost/MSE:  40051.881


## Question-4

In [106]:
import numpy as np
import copy, sys
import pandas as pd

def calculate_f(x, weights, y):
    """
    x.shape = (N,1)
    w.shape = (3,1)
    """
    x = np.hstack([x**2, x, np.ones(x.shape,dtype=np.float64)])
    return ((np.dot(x,weights) - y)**2).sum() / (2 * x.shape[0])

def grad_f(x:np.array, weights:np.array, y:np.array) -> np.array:
    new_x = np.hstack([x**2, x, np.ones(x.shape,dtype=np.float64)])
    common_term = (np.dot(new_x,weights) - y) / x.shape[0]
    first_term = (x ** 2 * common_term).sum()
    second_term = (x * common_term).sum()
    third_term = (common_term).sum()
    return np.array([[first_term],[second_term],[third_term]],dtype=np.float64)


beta1 = 1e-4
beta2 = 0.9
epsilon = 0.01
r = 0.5
w_0 = np.array([[0],[0],[0]], dtype=np.float64)
x = data[:,:-1]
y = data[:,-1:]
w, iterations = find_w1(w_0, x, y, beta1, beta2, epsilon, r)
# print('\n')
print(w)
print(f"total_iterations: {iterations}")
print(calculate_f(x, w, y))

[[-0.00485504]
 [ 3.67176844]
 [ 0.07355137]]
total_iterations: 82
2.0027581447644307


## Question-5

In [107]:
batch_size = 1
w_0 = np.array([[0],[0],[0]], dtype=np.float64)
x = data[:,:-1]
y = data[:,-1:]
data_loader = return_dataset(x, y, batch_size)
iteration = 0
while calculate_f(x, w_0, y) > 2.003:
    for idx, (dx,dy) in enumerate(data_loader):
        cur_w = find_w2(w_0, dx, dy, iteration)
        iteration += 1
        if calculate_f(x, cur_w, y) < 2.003:
            w_0 = cur_w
            break
        if calculate_f(x, cur_w, y) < calculate_f(x, w_0, y):
            w_0 = cur_w
    if iteration == 20000:
        break
    data_loader = return_dataset(x, y, batch_size)

print(w_0)
print(f"total_iterations: {iteration}")
print(calculate_f(x, w_0, y))

[[-0.00443891]
 [ 3.65458596]
 [ 0.0787922 ]]
total_iterations: 187
2.002913879428597


## Question-6

In [108]:
def cal_cost(theta, X, y):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1 / (2 * m)) * np.sum(np.square(predictions - y))
    return cost

def minibatch_gradient_descent(X, y, theta, learning_rate=0.01, iterations=10, batch_size=10):
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m / batch_size)
    for it in range(iterations):
        cost = 0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in range(0, m, batch_size):
            X_i = X[i:i + batch_size]
            y_i = y[i:i + batch_size]
            X_i = np.c_[np.ones(len(X_i)), X_i, X_i ** 2]  # Adding X^2 as a feature
            prediction = np.dot(X_i, theta)
            theta = theta - (1 / m) * learning_rate * (X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta, X_i, y_i)
        cost_history[it] = cost
    return theta, cost_history

# Example usage
lr = 0.05
n_iter = 1000
theta = np.random.randn(3, 1)  # Initialize theta with appropriate dimensions
theta, cost_history = minibatch_gradient_descent(x, y, theta, lr, n_iter)
print('Theta0:          {:0.3f},\nTheta1:          {:0.3f},\nTheta2:          {:0.3f}'.format(theta[0][0], theta[1][0], theta[2][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))


Theta0:          0.074,
Theta1:          3.672,
Theta2:          -0.003
Final cost/MSE:  400.502


### Question-7

In [109]:
db = pd.read_csv("diabetes2.csv")
db = np.array(db)
y = db[:,-1:]
X=db[:,:-1]
print("X shape:", X.shape)  # Should be (number_of_samples, 8)
print("y shape:", y.shape)  # Should be (number_of_samples, 1)

X shape: (768, 8)
y shape: (768, 1)


stocastic descent gradient

In [110]:
import numpy as np

def cal_cost(theta,X,y):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    return cost

def stocashtic_gradient_descent(X, y, theta, learning_rate=0.01, iterations=10):
    m = len(y)
    cost_history = np.zeros(iterations)
    for it in range(iterations):
        cost = 0.0
        for i in range(m):
            rand_ind = np.random.randint(0, m)
            X_i = X[rand_ind, :].reshape(1, X.shape[1])
            y_i = y[rand_ind].reshape(1, 1)
            prediction = np.dot(X_i, theta)
            theta = theta - (1 / m) * learning_rate * (X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta, X_i, y_i)
        cost_history[it] = cost
    return theta, cost_history

lr = 0.001
n_iter = 1000
theta = np.zeros((9,1))
X_b = np.c_[np.ones((len(X),1)),X]
theta,cost_history = stocashtic_gradient_descent(X_b,y,theta,lr,n_iter)
print('Theta0:          {:0.3f},\nTheta1:          {:0.3f},\nTheta2:          {:0.3f},\nTheta3:          {:0.3f},\nTheta4:          {:0.3f},\nTheta5:          {:0.3f},\nTheta6:          {:0.3f},\nTheta7:          {:0.3f},\nTheta8:          {:0.3f}'.format(theta[0][0],theta[1][0],theta[2][0],theta[3][0],theta[4][0],theta[5][0],theta[6][0],theta[7][0],theta[8][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

Theta0:          -0.024,
Theta1:          0.024,
Theta2:          0.004,
Theta3:          -0.005,
Theta4:          0.000,
Theta5:          0.000,
Theta6:          0.004,
Theta7:          0.009,
Theta8:          -0.001
Final cost/MSE:  63.805


Mini-batch descent gradient


In [111]:


# Define the gradient descent code
def cal_cost(theta, X, y):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1 / (2 * m)) * np.sum(np.square(predictions - y))
    return cost

def minibatch_gradient_descent(X,y,theta,learning_rate=0.01, iterations=10, batch_size =20):
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)
    for it in range(iterations):
        cost =0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]
            X_i = np.c_[np.ones(len(X_i)),X_i]
            prediction = np.dot(X_i,theta)
            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost
    return theta, cost_history

lr = 0.001
n_iter = 1000
theta = np.zeros((9,1))
theta,cost_history = minibatch_gradient_descent(X,y,theta,lr,n_iter)
print('Theta0:          {:0.3f},\nTheta1:          {:0.3f},\nTheta2:          {:0.3f},\nTheta3:          {:0.3f},\nTheta4:          {:0.3f},\nTheta5:          {:0.3f},\nTheta6:          {:0.3f},\nTheta7:          {:0.3f},\nTheta8:          {:0.3f}'.format(theta[0][0],theta[1][0],theta[2][0],theta[3][0],theta[4][0],theta[5][0],theta[6][0],theta[7][0],theta[8][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

Theta0:          -0.024,
Theta1:          0.024,
Theta2:          0.004,
Theta3:          -0.005,
Theta4:          0.000,
Theta5:          -0.000,
Theta6:          0.005,
Theta7:          0.010,
Theta8:          -0.001
Final cost/MSE:  3.338
